This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.2_tutorials/wallaroo-run-anywhere/inference/arm/wallaroo-arm-byop-vgg16).

## Model Upload API Error Message Tests


## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import wallaroo

from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework
import base64
import requests
import time
import json

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more details on logging in through Wallaroo, see the [Wallaroo SDK Essentials Guide: Client Connection](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client()



Please log into the following URL in a web browser:

	https://staging.wallaroo.dev/auth/realms/master/device?user_code=GEOC-JPXL

Login successful!


### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.



In [3]:
workspace_name = f'vgg16-clustering-workspace-error-api'


### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline that is used to deploy our arbitrary Python model.

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)


{'name': 'vgg16-clustering-workspace-error-api', 'id': 147, 'archived': False, 'created_by': '47ae86dc-efa5-4357-958d-497def7dfeba', 'created_at': '2025-12-18T23:14:18.024532+00:00', 'models': [{'name': 'api-byop-error-requirements-empty', 'versions': 2, 'owner_id': '""', 'last_update_time': datetime.datetime(2025, 12, 22, 18, 50, 52, 383998, tzinfo=tzutc()), 'created_at': datetime.datetime(2025, 12, 22, 17, 50, 28, 692621, tzinfo=tzutc())}, {'name': 'api-byop-error-demo-unmodified', 'versions': 3, 'owner_id': '""', 'last_update_time': datetime.datetime(2025, 12, 22, 18, 49, 12, 928244, tzinfo=tzutc()), 'created_at': datetime.datetime(2025, 12, 22, 17, 38, 10, 206882, tzinfo=tzutc())}], 'pipelines': []}

Define standard parameters

In [5]:
input_schema = pa.schema([
    pa.field('images', pa.list_(
        pa.list_(
            pa.list_(
                pa.int64(),
                list_size=3
            ),
            list_size=32
        ),
        list_size=32
    )),
])

output_schema = pa.schema([
    pa.field('predictions', pa.int64()),
])

encoded_input_schema = base64.b64encode(
                bytes(input_schema.serialize())
            ).decode("utf8")

encoded_output_schema = base64.b64encode(
                bytes(output_schema.serialize())
            ).decode("utf8")

framework = 'custom'

### Test 1:  Normal Model

Unmodified model to show it can upload successfully without issue.



In [6]:
workspace.id()

147

In [9]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'api-byop-error-demo-unmodified'
model_file_name = './models/byop-vgg-unmodified.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)

'Uploaded Model Name: api-byop-error-demo-unmodified.'

{'insert_models': {'returning': [{'models': [{'id': 274}]}]}}

In [10]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": response['insert_models']['returning'][0]['models'][0]['id']
}

response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
display(response)

{'model_version': {'model_version': {'name': 'api-byop-error-demo-unmodified',
   'visibility': 'private',
   'workspace_id': 147,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-vgg-unmodified.zip/model'}}},
   'id': 274,
   'image_path': None,
   'status': 'error',
   'task_id': 'f9b7e75c-13e9-4876-9f15-a2da9b8e896a',
   'file_info': {'version': '27fe6b1c-a93b-4f58-95d5-ab557773e9d2',
    'sha': '79472d4b9652937c15e1887478c23effe1160363b985920e083fec72868810f4',
    'file_name': './models/byop-vgg-unmodified.zip',
    'size': 54695306},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-22T18:49:12.928244+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 442,
   'model_version_id': 274,
   'runtime': 'flight',
   'filter_threshold': None,
   'tensor_fields': None,
  

### Test 2:  Requirements.txt is missing

In the following test, the Requirements.txt file is empty.

zip file contents:

```bash
Archive:  byop-vgg16-requirements-empty.zip
  inflating: custom_inference.py     
  inflating: feature_extractor.h5    
  inflating: kmeans.pkl      
```


In [11]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'api-byop-error-requirements-empty'
model_file_name = './models/byop-vgg16-requirements-empty.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}



headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)

'Uploaded Model Name: api-byop-error-requirements-empty.'

{'insert_models': {'returning': [{'models': [{'id': 275}]}]}}

In [12]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": response['insert_models']['returning'][0]['models'][0]['id']
}

response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
display(response)

{'model_version': {'model_version': {'name': 'api-byop-error-requirements-empty',
   'visibility': 'private',
   'workspace_id': 147,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-vgg16-requirements-empty.zip/model'}}},
   'id': 275,
   'image_path': None,
   'status': 'pending_load_container',
   'task_id': '2de8e446-f04b-4e38-82f4-5cd18ce2cd19',
   'file_info': {'version': '503bc84c-d325-46c1-9f2f-3df27d150fcd',
    'sha': '47e1962acba0a53481d9e3929265633f3e8fc0e7b067a63b1e79140d94f3a8a8',
    'file_name': './models/byop-vgg16-requirements-empty.zip',
    'size': 54694320},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-22T18:50:52.383998+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 443,
   'model_version_id': 275,
   'runtime': 'flight',
   'filter_thresh

In [7]:
no_requirements = wl.get_model("api-byop-error-requirements-empty")
no_requirements

Name,api-byop-error-requirements-empty
Version,503bc84c-d325-46c1-9f2f-3df27d150fcd
File Name,./models/byop-vgg16-requirements-empty.zip
SHA,47e1962acba0a53481d9e3929265633f3e8fc0e7b067a63b1e79140d94f3a8a8
Status,error
Error Summary,Framework.CUSTOM requires a requirements.txt to be included at the root level within the uploaded .zip file
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2025-22-Dec 18:51:08
Workspace id,147


In [15]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_upload_logs"

data = {
  "model_version_id": 275
}

response = requests.post(endpoint, 
                         json=data, 
                         headers=headers, 
                         verify=True).json()
display(response)

{'logs': []}

### A .py file without an Inference class 



In [14]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'api-byop-error-inference-missing'
model_file_name = './models/byop-vgg16-inference-missing.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}



headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)

'Uploaded Model Name: api-byop-error-inference-missing.'

{'insert_models': {'returning': [{'models': [{'id': 279}]}]}}

In [15]:
model_version_id = response['insert_models']['returning'][0]['models'][0]['id']

In [17]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": model_version_id
}

response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
display(response)

{'model_version': {'model_version': {'name': 'api-byop-error-inference-missing',
   'visibility': 'private',
   'workspace_id': 147,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-vgg16-inference-missing.zip/model'}}},
   'id': 279,
   'image_path': None,
   'status': 'attempting_load_container',
   'task_id': '4230e70c-d764-4a8e-936f-d3f17a0a10ce',
   'file_info': {'version': 'b29a0e3b-eaeb-448c-a933-6324ee3f3ac1',
    'sha': '9cad003c9bf8d538496050a20d73eb52d62d10de0d2fccb6bec52cdde9e8e677',
    'file_name': './models/byop-vgg16-inference-missing.zip',
    'size': 54693828},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-22T19:00:03.346416+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 447,
   'model_version_id': 279,
   'runtime': 'flight',
   'filter_thresh

In [18]:
inference_missing = wl.get_model("api-byop-error-inference-missing")
inference_missing

Name,api-byop-error-inference-missing
Version,b29a0e3b-eaeb-448c-a933-6324ee3f3ac1
File Name,./models/byop-vgg16-inference-missing.zip
SHA,9cad003c9bf8d538496050a20d73eb52d62d10de0d2fccb6bec52cdde9e8e677
Status,attempting_load_container
Error Summary,None
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2025-22-Dec 19:00:12
Workspace id,147


### Test 3: An Inference class without expected_model_types method

In [19]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'no-expected-model-types'
model_file_name = './models/byop-vgg16-no-expected-model-types.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)

'Uploaded Model Name: no-expected-model-types.'

{'insert_models': {'returning': [{'models': [{'id': 280}]}]}}

In [ ]:
missing_model_types_id = response['insert_models']['returning'][0]['models'][0]['id']

In [ ]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": missing_model_types_id
}

response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
display(response)

{'model_version': {'model_version': {'name': 'no-expected-model-types',
   'visibility': 'private',
   'workspace_id': 147,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-vgg16-no-expected-model-types.zip/model'}}},
   'id': 280,
   'image_path': None,
   'status': 'error',
   'task_id': '97a94279-e8fe-45ab-b8f3-e8647d164892',
   'file_info': {'version': '55f5099c-0efc-49b4-8766-030e29317877',
    'sha': '1b99d478e8483c306fbbc4fc6f51b4fa5ad8831a1667960d307fb6ed3548fdf3',
    'file_name': './models/byop-vgg16-no-expected-model-types.zip',
    'size': 54694496},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-22T19:29:09.467166+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 448,
   'model_version_id': 280,
   'runtime': 'flight',
   'filter_threshold': None,
   't

In [26]:
missing_expected_model_types = wl.get_model("no-expected-model-types")
missing_expected_model_types

Name,no-expected-model-types
Version,55f5099c-0efc-49b4-8766-030e29317877
File Name,./models/byop-vgg16-no-expected-model-types.zip
SHA,1b99d478e8483c306fbbc4fc6f51b4fa5ad8831a1667960d307fb6ed3548fdf3
Status,error
Error Summary,null
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2025-22-Dec 19:29:45
Workspace id,147


### An Inference class without a predict method


In [27]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'missing-precict'
model_file_name = './models/byop-vgg16-missing-predict.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)

'Uploaded Model Name: missing-precict.'

{'insert_models': {'returning': [{'models': [{'id': 281}]}]}}

In [28]:
model_version_id_missing_predict = response['insert_models']['returning'][0]['models'][0]['id']

In [34]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": model_version_id_missing_predict
}

response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
display(response)

{'model_version': {'model_version': {'name': 'missing-precict',
   'visibility': 'private',
   'workspace_id': 147,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-vgg16-missing-predict.zip/model'}}},
   'id': 281,
   'image_path': None,
   'status': 'error',
   'task_id': '6a250eab-4d42-4283-88b6-9d43f95e784e',
   'file_info': {'version': '476e6f2c-699e-43e5-9f40-622149cc1d94',
    'sha': '83d8d4ca0e2095bd911610f4bfc6c4613495276936342dd2767826be86bc91b8',
    'file_name': './models/byop-vgg16-missing-predict.zip',
    'size': 54694186},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-22T19:38:53.978714+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 449,
   'model_version_id': 281,
   'runtime': 'flight',
   'filter_threshold': None,
   'tensor_fields': None,
   

In [33]:
missing_expected_model_types = wl.get_model("missing-precict")
missing_expected_model_types

Name,missing-precict
Version,476e6f2c-699e-43e5-9f40-622149cc1d94
File Name,./models/byop-vgg16-missing-predict.zip
SHA,83d8d4ca0e2095bd911610f4bfc6c4613495276936342dd2767826be86bc91b8
Status,error
Error Summary,null
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2025-22-Dec 19:39:32
Workspace id,147


### A Predict method that does not accept a dictionary of numpy arrays

In [35]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-no-accept-numpy'
model_file_name = './models/byop-no-accept-numpy.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-no-accept-numpy.'

{'insert_models': {'returning': [{'models': [{'id': 282}]}]}}

In [36]:
model_no_accept_numpy = response['insert_models']['returning'][0]['models'][0]['id']

In [38]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": model_no_accept_numpy
}

response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
display(response)

{'model_version': {'model_version': {'name': 'byop-no-accept-numpy',
   'visibility': 'private',
   'workspace_id': 147,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-no-accept-numpy.zip/model'}}},
   'id': 282,
   'image_path': None,
   'status': 'error',
   'task_id': 'ff5bb902-80d7-4c47-8300-66ba96db4f2c',
   'file_info': {'version': 'ee8623bf-9fb2-4fee-b4bf-dc97ab4902fa',
    'sha': '492c38a89d0f91d91c6bac0f5a267c567965ff7e58a16e9cc9f337ec726fb65c',
    'file_name': './models/byop-no-accept-numpy.zip',
    'size': 54694520},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-22T19:45:23.915023+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 450,
   'model_version_id': 282,
   'runtime': 'flight',
   'filter_threshold': None,
   'tensor_fields': None,
   'input_

In [39]:
model_no_accept_numpy = wl.get_model("byop-no-accept-numpy")
model_no_accept_numpy

Name,byop-no-accept-numpy
Version,ee8623bf-9fb2-4fee-b4bf-dc97ab4902fa
File Name,./models/byop-no-accept-numpy.zip
SHA,492c38a89d0f91d91c6bac0f5a267c567965ff7e58a16e9cc9f337ec726fb65c
Status,error
Error Summary,null
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2025-22-Dec 19:46:09
Workspace id,147


### A Predict method that does not return a dictionary of numpy arrays

In [40]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-no-return-numpy'
model_file_name = './models/byop-no-return-numpy.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-no-return-numpy.'

{'insert_models': {'returning': [{'models': [{'id': 283}]}]}}

In [41]:
model_no_return_numpy_id = response['insert_models']['returning'][0]['models'][0]['id']

In [44]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": model_no_return_numpy_id
}

response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
display(response)

{'model_version': {'model_version': {'name': 'byop-no-return-numpy',
   'visibility': 'private',
   'workspace_id': 147,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-no-return-numpy.zip/model'}}},
   'id': 283,
   'image_path': None,
   'status': 'error',
   'task_id': 'd98eb63f-ebdf-4404-8f89-7bb87559820b',
   'file_info': {'version': '4a663a1f-51df-49c3-b955-66490ac0c1ff',
    'sha': 'b2fdd794d4f057d19daece32121f59c12f98854d75bd57b2bea3e54dbd90b4da',
    'file_name': './models/byop-no-return-numpy.zip',
    'size': 54694513},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-22T19:50:38.857627+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 451,
   'model_version_id': 283,
   'runtime': 'flight',
   'filter_threshold': None,
   'tensor_fields': None,
   'input_

In [45]:
model_no_return_numpy = wl.get_model("byop-no-return-numpy")
model_no_return_numpy

Name,byop-no-return-numpy
Version,4a663a1f-51df-49c3-b955-66490ac0c1ff
File Name,./models/byop-no-return-numpy.zip
SHA,b2fdd794d4f057d19daece32121f59c12f98854d75bd57b2bea3e54dbd90b4da
Status,error
Error Summary,null
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2025-22-Dec 19:51:46
Workspace id,147


In [46]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-vgg16-missing-model-method'
model_file_name = './models/byop-vgg16-missing-model-method.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-vgg16-missing-model-method.'

{'insert_models': {'returning': [{'models': [{'id': 284}]}]}}

In [50]:
byop_missing_model_method_id = response['insert_models']['returning'][0]['models'][0]['id']

In [51]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": byop_missing_model_method_id
}

response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
display(response)

{'model_version': {'model_version': {'name': 'byop-vgg16-missing-model-method',
   'visibility': 'private',
   'workspace_id': 147,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-vgg16-missing-model-method.zip/model'}}},
   'id': 284,
   'image_path': None,
   'status': 'attempting_load_container',
   'task_id': '648c235d-9616-456f-bfc7-23f7499732ca',
   'file_info': {'version': '19205b38-2f2e-4680-ab9d-e7e612395b6b',
    'sha': '6ad88b44cd6053adffc7e54f2faae7081147cda503d2a514465e4b3c5ca82ab7',
    'file_name': './models/byop-vgg16-missing-model-method.zip',
    'size': 54694496},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-22T19:56:08.184337+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 452,
   'model_version_id': 284,
   'runtime': 'flight',
   'filter_t

In [56]:
byop_no_model_method = wl.get_model("byop-vgg16-missing-model-method")
byop_no_model_method

Name,byop-vgg16-missing-model-method
Version,19205b38-2f2e-4680-ab9d-e7e612395b6b
File Name,./models/byop-vgg16-missing-model-method.zip
SHA,6ad88b44cd6053adffc7e54f2faae7081147cda503d2a514465e4b3c5ca82ab7
Status,error
Error Summary,null
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2025-22-Dec 19:57:37
Workspace id,147


### A .py file without an InferenceBuilder class

In [58]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-vgg16-missing-inference-builder'
model_file_name = './models/byop-missing-inferencebilder.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-vgg16-missing-inference-builder.'

{'insert_models': {'returning': [{'models': [{'id': 285}]}]}}

In [59]:
byop_missing_inference_builder_id = response['insert_models']['returning'][0]['models'][0]['id']

In [62]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": byop_missing_inference_builder_id
}

response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
display(response)

{'model_version': {'model_version': {'name': 'byop-vgg16-missing-inference-builder',
   'visibility': 'private',
   'workspace_id': 147,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-missing-inferencebilder.zip/model'}}},
   'id': 285,
   'image_path': None,
   'status': 'attempting_load_container',
   'task_id': '797eab58-1d19-47d8-b65d-7346b735694e',
   'file_info': {'version': '271ef578-6b93-4a74-b7bd-a767ac6145b9',
    'sha': '09c4b9917e106a1fb4412b829b78fe2ccb4d9841fead95462665373a0ba8a600',
    'file_name': './models/byop-missing-inferencebilder.zip',
    'size': 54694110},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-22T20:09:51.931395+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 453,
   'model_version_id': 285,
   'runtime': 'flight',
   'filter_th

In [63]:
byop_missing_inference_builder = wl.get_model("byop-vgg16-missing-inference-builder")
byop_missing_inference_builder

Name,byop-vgg16-missing-inference-builder
Version,271ef578-6b93-4a74-b7bd-a767ac6145b9
File Name,./models/byop-missing-inferencebilder.zip
SHA,09c4b9917e106a1fb4412b829b78fe2ccb4d9841fead95462665373a0ba8a600
Status,attempting_load_container
Error Summary,None
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2025-22-Dec 20:09:59
Workspace id,147


### An InferenceBuilder class without an inference method

In [ ]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-missing-inference-method'
model_file_name = './models/byop-inference-builder-missing-inference-method.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-missing-inference-method.'

{'insert_models': {'returning': [{'models': [{'id': 286}]}]}}

In [65]:
byop_missing_inference_method_id = response['insert_models']['returning'][0]['models'][0]['id']

In [70]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": byop_missing_inference_method_id
}

response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
display(response)

{'model_version': {'model_version': {'name': 'byop-missing-inference-method',
   'visibility': 'private',
   'workspace_id': 147,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-missing-inference-method.zip/model'}}},
   'id': 286,
   'image_path': None,
   'status': 'pending_load_container',
   'task_id': '65389fcb-1876-4269-a75c-ce3849be5eb6',
   'file_info': {'version': '3620e2ea-e710-4bff-b727-cfeff6097f14',
    'sha': '3e8ec360d2135e8301ab8bc3c27f70189be3a373d2dc644262bbce43350b2798',
    'file_name': './models/byop-missing-inference-method.zip',
    'size': 54694503},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-22T20:15:27.151798+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 454,
   'model_version_id': 286,
   'runtime': 'flight',
   'filter_threshold'

In [69]:
byop_missing_inference_method_model = wl.get_model("byop-missing-inference-method")
byop_missing_inference_builder

Name,byop-vgg16-missing-inference-builder
Version,271ef578-6b93-4a74-b7bd-a767ac6145b9
File Name,./models/byop-missing-inferencebilder.zip
SHA,09c4b9917e106a1fb4412b829b78fe2ccb4d9841fead95462665373a0ba8a600
Status,error
Error Summary,null
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2025-22-Dec 20:11:53
Workspace id,147


### An InferenceBuilder class without a create method

In [71]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-missing-create-method'
model_file_name = './models/byop-missing-create-method.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-missing-create-method.'

{'insert_models': {'returning': [{'models': [{'id': 287}]}]}}

In [72]:
byop_missing_create_method_id = response['insert_models']['returning'][0]['models'][0]['id']

In [73]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": byop_missing_create_method_id
}

response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
display(response)

{'model_version': {'model_version': {'name': 'byop-missing-create-method',
   'visibility': 'private',
   'workspace_id': 147,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-missing-create-method.zip/model'}}},
   'id': 287,
   'image_path': None,
   'status': 'pending_load_container',
   'task_id': '401229bf-1e5e-4385-bc3d-5c8097ac3ca3',
   'file_info': {'version': 'dcefc5e5-4ecd-442d-903f-c5690d6022f0',
    'sha': 'c6769ec3ff258e4e5c7b4cb44e28b46286c85aad1ca2651d4bbcec6e5d073f49',
    'file_name': './models/byop-missing-create-method.zip',
    'size': 54694274},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-22T20:20:22.35894+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 455,
   'model_version_id': 287,
   'runtime': 'flight',
   'filter_threshold': None,
  

In [75]:
byop_missing_inference_method_model = wl.get_model("byop-missing-create-method")
byop_missing_inference_builder

Name,byop-vgg16-missing-inference-builder
Version,271ef578-6b93-4a74-b7bd-a767ac6145b9
File Name,./models/byop-missing-inferencebilder.zip
SHA,09c4b9917e106a1fb4412b829b78fe2ccb4d9841fead95462665373a0ba8a600
Status,error
Error Summary,null
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2025-22-Dec 20:11:53
Workspace id,147


### Package dependencies fail to install

In [76]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-bad-dependancies'
model_file_name = './models/byop-bad-dependancies.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-bad-dependancies.'

{'insert_models': {'returning': [{'models': [{'id': 289}]}]}}

In [77]:
byop_bad_dependancies_id = response['insert_models']['returning'][0]['models'][0]['id']

In [80]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": byop_bad_dependancies_id
}

response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
display(response)

{'model_version': {'model_version': {'name': 'byop-bad-dependancies',
   'visibility': 'private',
   'workspace_id': 147,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-bad-dependancies.zip/model'}}},
   'id': 289,
   'image_path': None,
   'status': 'error',
   'task_id': 'd9909e8b-d25a-48f3-b574-60781dd8e48b',
   'file_info': {'version': '9c758f65-be8d-4aac-a65f-7a563fd3dcf8',
    'sha': 'd080aba4c2091fe6f3c252377c06b3eaed9726b2e6e48087fd4fb76f112f7b3c',
    'file_name': './models/byop-bad-dependancies.zip',
    'size': 54694526},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-22T20:25:06.339283+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 457,
   'model_version_id': 289,
   'runtime': 'flight',
   'filter_threshold': None,
   'tensor_fields': None,
   'inp

In [81]:
byop_bad_dependancies = wl.get_model("byop-bad-dependancies")
byop_bad_dependancies

Name,byop-bad-dependancies
Version,9c758f65-be8d-4aac-a65f-7a563fd3dcf8
File Name,./models/byop-bad-dependancies.zip
SHA,d080aba4c2091fe6f3c252377c06b3eaed9726b2e6e48087fd4fb76f112f7b3c
Status,error
Error Summary,Could not pip install user provided requirements.txt
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2025-22-Dec 20:25:25
Workspace id,147
